In [ ]:
import random
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from google.colab import drive
import numpy as np

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)  # 감성 라벨이 긍정/부정 두개이므로 num_labels=2으로 설정

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
# 모델 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/L25_drop02_10epoch_sche0_4_best_model.pth'))
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
def predict_sentiment(review_text, model, tokenizer):
    # 리뷰 텍스트를 토크나이저를 통해 인코딩
    inputs = tokenizer.encode_plus(
        review_text,
        truncation=True,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    # 모델의 예측 생성
    with torch.no_grad():
        outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    # 가장 확률이 높은 결과를 선택
    _, prediction = torch.max(outputs.logits, dim=1)
    # 예측 결과 반환 (0: 부정, 1: 긍정)
    return "긍정" if prediction.item() == 1 else "부정"

In [ ]:
while True:
    # 사용자로부터 텍스트 입력 받기
    review_text = input("리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): ")
    if review_text.lower() == 'exit':
        break
    # 예측
    print(predict_sentiment(review_text, model, tokenizer))


리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 7스킨
긍정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 다크닝
부정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 당김
부정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 도톰
긍정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 밀착력
긍정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 발림성
긍정
리뷰 텍스트를 입력하세요 (종료하려면 'exit' 입력): 백탁
부정
